# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'human_nontata_promoters'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "human_enhancers_cohn"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_enhancers_cohn 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf

from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded('human_nontata_promoters'):
    download_dataset('human_nontata_promoters')

2021-10-23 00:02:33.493913: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-23 00:02:33.493928: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2021-10-23 00:02:36.263831: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-23 00:02:36.263882: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michle): /proc/driver/nvidia/version does not exist
2021-10-23 00:02:36.266072: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
info(DATASET)

Dataset `human_enhancers_cohn` has 2 classes: negative, positive.

All lenghts of genomic intervals equals 500.

Totally 27791 sequences have been found, 20843 for training and 6948 for testing.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:96: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


,train,test
negative,10422,3474
positive,10421,3474


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 20843 files belonging to 2 classes.


## Text vectorization

In [7]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
#vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

2021-10-23 00:02:37.021426: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


['', '[UNK]', 't', 'a', 'c', 'g']

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [10]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


  1/326 [..............................] - ETA: 4:11 - loss: 0.7613 - binary_accuracy: 0.4688

  3/326 [..............................] - ETA: 8s - loss: 0.7255 - binary_accuracy: 0.5000  

  5/326 [..............................] - ETA: 8s - loss: 0.7006 - binary_accuracy: 0.5281

  7/326 [..............................] - ETA: 8s - loss: 0.6763 - binary_accuracy: 0.5647

...

319/326 [============================>.] - ETA: 0s - loss: 0.4809 - binary_accuracy: 0.7662

321/326 [============================>.] - ETA: 0s - loss: 0.4810 - binary_accuracy: 0.7659

323/326 [============================>.] - ETA: 0s - loss: 0.4811 - binary_accuracy: 0.7660

325/326 [============================>.] - ETA: 0s - loss: 0.4817 - binary_accuracy: 0.7656

326/326 [==============================] - 10s 32ms/step - loss: 0.4817 - binary_accuracy: 0.7657


## Evaluation on the test set

In [11]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 6948 files belonging to 2 classes.


In [12]:
model.evaluate(test_ds)

  1/109 [..............................] - ETA: 24s - loss: 0.5264 - binary_accuracy: 0.6875

  6/109 [>.............................] - ETA: 1s - loss: 0.5867 - binary_accuracy: 0.6927 

 11/109 [==>...........................] - ETA: 1s - loss: 0.5841 - binary_accuracy: 0.7003

 15/109 [===>..........................] - ETA: 1s - loss: 0.5813 - binary_accuracy: 0.6906

 19/109 [====>.........................] - ETA: 1s - loss: 0.5702 - binary_accuracy: 0.6949

...

 95/109 [=========================>....] - ETA: 0s - loss: 0.5745 - binary_accuracy: 0.7122

100/109 [==========================>...] - ETA: 0s - loss: 0.5783 - binary_accuracy: 0.7097

105/109 [===========================>..] - ETA: 0s - loss: 0.5796 - binary_accuracy: 0.7085

109/109 [==============================] - 2s 12ms/step - loss: 0.5809 - binary_accuracy: 0.7078


[0.5808840394020081, 0.7078295946121216]